# Import

In [252]:
import numpy as np
from random import choice, choices, randint,random, uniform
import copy
import math
from dataclasses import dataclass

# Operations
Operations divided based on the arity in order to make more consistent program generation during initialization. Indeed, we didn't want that functions such as sqrt,cos or sin contained only constants at the beginning. Those are values that can be 'easily' recreatted with a different constant, so it only increases the depth of the tree and the complexity of the formula without a real advantage.

Moreover, a weight empirically set through different trials has been associated to all the operations. This is done because, in particular for binary operations, most useful ones are sum and subtraction, while, for instance, the power is used less and can create more complex formulas that are not so common and useful in the examples provided. 

Also the max depth of the tree is set in order to avoid blooding, a problem that mainly exists because of the crossover and causes the formula to grow a lot without real changes into fitness

In [253]:
#The depth of a tree cannot be greater than 10 OVERALL
MAX_TREE_DEPTH = 10

#Operations with 1 or 2 arguments divided
#Added a field of probabilities: sum and subtraction should be more likely to be chosen
OPERATIONS_BINARY = [ 
    (np.add, 2, "({} + {})",0.20),
    (np.subtract, 2, "({} - {})",0.20),
    (np.divide, 2, "({} / {})",0.15),
    (np.multiply, 2, "({} * {})",0.15),   
    (np.power, 2, "({} ^ {})",0.04),
]

BINARY_WEIGHTS = [op[3] for op in OPERATIONS_BINARY]

OPERATIONS_UNARY = [ 
    (np.sin, 1, "sin({})",0.05),
    (np.cos, 1, "cos({})",0.05),
    (np.tan, 1, "tan({})",0.04),
    (np.exp, 1, "exp({})",0.04),
    (np.log, 1, "log({})",0.04),
    (np.sqrt, 1, "sqrt({})",0.4),
]
UNARY_WEIGHTS = [op[3] for op in OPERATIONS_UNARY]
OPERATIONS = OPERATIONS_BINARY + OPERATIONS_UNARY
WEIGHTS = BINARY_WEIGHTS + UNARY_WEIGHTS    


# Program evaluation
We need a function that, given the genotype, provides us with the output numerical output given by the input applied to the function. Thus, it has to receive as arguments the program (that represents the function) and the input vector with all the dimensions. Then, it simply recursively travels the tree and compute all the operations.

In [254]:
def evaluate_program(program, x):
    if isinstance(program, str):  # Leaf node
        #If it's a leaf, it could be a costant or a variable
        if program[0] == 'x':
            return x[int(program[2:-1])]
        else:
            return float(program)
    elif isinstance(program, list): 
        op = next(op for op, _, symbol,p in OPERATIONS if symbol == program[0])
        args = [evaluate_program(child, x) for child in program[1:]]
        try:
            return op(*args)
        except ZeroDivisionError:
            return np.inf

# Mutation utility functions

Set of utility functions used in the mutation and crossover functions. A brief explanation of all of them is provided at the definition.

In [255]:

def get_subtree_points_recursive(prog, path='', index=0, result=None):
    """ Get the points of the subtrees in the program. """
    if result is None:
        result = []
    
    if isinstance(prog, list) and len(prog) > 1:  
        if path:
            result.append(path)
        for i, node in enumerate(prog):
            if isinstance(node, list):  
                new_path = f"{path}[{i}]" if path else f"[{i}]"
                get_subtree_points_recursive(node, new_path, i, result)
    return result

def access_node_by_path(prog, path):
    """Access a node in the program by its path."""
    indices = [int(p.strip('][')) for p in path.split('][') if p]
    current = prog
    for index in indices:
        current = current[index]
    return current


def find_variable_indices(node, result=None):
    """Find the indices of the variables in the program."""
    if result is None:
        result = set()

    if isinstance(node, list):
        for child in node:
            find_variable_indices(child, result)
    
    elif isinstance(node, str) and node.startswith('x['):
        result.add(int(node[2:-1]))

    return result

def set_subtree_at_path(program, path, new_subtree):
    """Set the new subtree at the specified path."""
    if path == '':
        return new_subtree
    current = program
    indices = [int(x) for x in path.strip('][').split('][')]
    for i in indices[:-1]:  
        current = current[i]
    current[indices[-1]] = new_subtree  
    return program


def swap_operation_at_path(program, path, new_op):
    """Swap the operation at the specified path."""
    current = program
    indices = [int(x) for x in path.strip('][').split('][')]
    for i in indices[:-1]: 
        current = current[i]
    if isinstance(current[indices[-1]], list):  
        current[indices[-1]][0] = new_op 
    return program

def cleaned_program(program):
    """Given a program of only costants, substitute it with a single value given by the computation of costants"""
    value = evaluate_program(program, [0])
    return str(value)

def safe_copy(obj):
    """Copy an object safely."""
    if isinstance(obj, list):
        return copy.deepcopy(obj)  
    elif isinstance(obj, str):
        return obj 
    else:
        raise TypeError("Unsupported type, only strings.")
    
def depth(program):
    """Compute the depth of the program (represented by a tree)"""
    if isinstance(program, str):
        return 1
    elif isinstance(program, list):
        return 1 + max(depth(child) for child in program[1:])
    
def get_subtree(program):
    """Get a random subtree from the program."""
    if isinstance(program, (str, int, float)):
        return program, None
    # Pick a random node
    choice = randint(0, len(program) - 1)
    if choice == 0:
        # Return the entire program
        return program, None
    else:
        # Recursion on one of the children
        subtree, parent = get_subtree(program[choice])
        return subtree, (program, choice)

# Genothype definition

The recursive function we've developed is designed to create complex mathematical formulas in a structured format, where each formula is represented as a list: `[operator, first_operand, second_operand]`. This format allows the recursive and hierarchical organization of operations, facilitating the computational evaluation and manipulation of the formula.

Key features of this function include:

1. **Use of Input Dimensions:** The function uses a list, `used_indices`, to track which input dimensions (e.g., `0,1`, etc.) have been utilized in the formula. This ensures comprehensive coverage of all available input variables (if the depth makes it possible), making the formula relevant to all dimensions of the input data.  

2. **Restrictions on Trigonometric Functions:** To enhance the mathematical sensibility of the formulas generated, the function includes a specific constraint regarding the nesting of trigonometric functions. Once a trigonometric function (such as `sin`, `cos`, etc.) is used, the function prohibits the inclusion of another trigonometric function within it. This constraint helps in preventing mathematically nonsensical expressions like `sin(cos(tan(x)))`, which, while computationally valid, may not be practically meaningful or may complicate the interpretation and analysis of the formula.
3. **Restrictions on operations with constants:** It is not allowed, in the current implementation, to add binary operations between constants. This is another kind of operation that makes the depth of the tree increase without the generation of a real meaningful formula.

This approach not only ensures that each formula is robust and contextually appropriate but also maintains clarity and reduces the computational redundancy that might arise from nested trigonometric operations. Such constraints are particularly important in scientific computing and simulations where the accuracy and interpretability of mathematical expressions are critical.



In [256]:
def random_program(depth,input_dim,unary=False, used_indices=None):
    if used_indices is None:
        used_indices = set()

    # Base case: generate a leaf node
    #If you already used all the variables, you can only use constants and return: don't place an operation between costants
    if depth == 0 or (random()<0.3) or len(used_indices) == input_dim:
        if len(used_indices) < input_dim and random()<0.9:
            available_indices = list(set(range(input_dim)) - used_indices)
            index = choice(available_indices)
            used_indices.add(index)
            return f"x[{index}]", used_indices
        else: #Only positive float between 0 and 10, no need to include negative ones, we have the subtraction operation
            return str(uniform(0, 10)), used_indices
           
    if(not unary): #Condition to check that we do not nest unary operations
        operations = OPERATIONS
        weights = WEIGHTS
    else:
        operations = OPERATIONS_BINARY
        weights = BINARY_WEIGHTS
    op, arity, symbol,p = choices(operations,weights=weights, k=1)[0]
    if(unary != True):
        unary = arity==1
    children = []
    for _ in range(arity):
        child, used_indices = random_program(depth - 1, input_dim, unary, used_indices)
        children.append(child)
    return [symbol] + children,used_indices

# Program generation for the initial population
This function, given the input dimension (the number of variables), generates a program that contains all of them once by generating one subtree for each dimension and combining all the subtrees together. This allows an initial population where there is a lot of genetic material related to all the dimensions of the problem and we can create more different combinations. 

In [257]:
def generate_program(input_dim):
    """Generate a random program given the input dimension. It ensures all dimensions are used once in the program."""
    programs = []
    for i in range(input_dim):
        flag = False #Evaluate that it does not only contain costants
        used_indices_local = set()
        used_indices_local = set(range(input_dim)) - {i}
        while(not flag):
            program, _ = random_program(2, input_dim,used_indices=used_indices_local)
            flag = len(find_variable_indices(program)) == 1
            if not flag:
                program = cleaned_program(program)
        programs.append(program)
    #Combine programs together through a binary operation
    program = programs[0]
    for i in range(1,len(programs)):
        op, arity, symbol,p = choices(OPERATIONS_BINARY,weights=BINARY_WEIGHTS, k=1)[0]
        program = [symbol] + [program, programs[i]]
    
    return program

## Individual

We used dataclass to store the fitness and the genome of each individual. This will allow us to avoid to recompute the fitness function for the same individual more than once.

In [258]:
@dataclass
class Individual:
    genome: list
    fitness : float = None

## Transform the program into a human readable function
Given the list representation of programs that we have introduced before, this function allows to rewrite it in a human readable way by simply recursively evaluating all nodes.

In [259]:
def program_to_string(program):
    if isinstance(program, str):  # leaf
        return program  
    elif isinstance(program, list):  
        try:
            _, _, symbol = next((op, arity, s) for op, arity, s,p in OPERATIONS if s == program[0])
        except StopIteration:
            raise ValueError(f"Not known operation: {program[0]}")
        
        children = [program_to_string(child) for child in program[1:]]
        
        return symbol.format(*children)

## Fitness function

# MAPE
The Mean Absolute Percentage Error (MAPE) is a statistical measure used to evaluate the accuracy of predictions in regression analysis. It quantifies the average magnitude of errors in a set of predictions, expressed as a percentage of the actual values. The formula for MAPE is the average of the absolute differences between the predicted and actual values, divided by the actual values, multiplied by 100 to convert it into a percentage. This metric is particularly useful because it provides a clear, interpretable measure of predictive error relative to the size of the numbers being predicted, making it ideal for comparing the accuracy of prediction models across different data scales

In [260]:
def mape(program, x, y):
    predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
    return  float(np.mean(np.abs((y - predictions) / y)) * 100)

# MSE 
Defined as the one used for the final evaluation by professors. 
We used MAPE only for printing it, the fitness function associated to the individual is the mse. Indeed, mse and mape do not vary in a consistent way. There are functions that decreases the mse while increasing the mape, and this can be dued in particular to the presence of 'outliers', which are values that reports a huge different with respect to the real value and count more in the percentage with respect to the mse.

Since the final evaluation on the quality of the work is done based on the mse, we decided to keep it as fitness. 

However, since the problems are on a different scale, the mse may be misleading: it is higher in some problems because the y reports high values so, even if the difference between the prediction and the real value is not so high in percentage, the mse is high. That's why we also printed mape to have an idea regarding the quality of the algorithm when doing experimentations.

In [261]:
def mse(program, x, y):
    predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
    return float(100*np.mean((y - predictions) ** 2))

In [262]:
def fitness_function(program, x, y):
    """Fitness evaluation for a program."""
    try:
        # Evaluation of the program
        predictions = np.array([evaluate_program(program, x_row) for x_row in x.T])
        if np.any(np.isnan(predictions)) or np.any(np.isinf(predictions)):
            return np.inf  # Penalize invalid programs 

        #Compute mse
        error = mse(program, x, y)
        if not math.isfinite(error):
            return np.inf
        #Evaluation of the complexity of a program removed, it is problem dependant, the depth of a program is limited anyway

    except Exception as e:
        # Penalize programs with errors 
        print(f"Error in program evaluation: {e}")
        return np.inf

    return error

## Mutation

The idea of the mutation is that recursively finds all the subtree of the tree. Casually select one of them. It checks which variables were in that subtree and generates a new tree containing (at least) the same variables to be added at that place: it ensures that we always consider all the dimensions of the input. 

If the considered sub tree do not contains any variable's dimension, we can return a constant value.

In [263]:
def mutate(program, input_dim, max_depth=3):
    """Mutation of a program."""
    mutant = copy.deepcopy(program)

    points = get_subtree_points_recursive(program)
    if not points:
        return generate_program(input_dim)

    point = choice(points)
    subtree = access_node_by_path(mutant, point)
    variable_set = find_variable_indices(subtree)
    variables = set(range(input_dim))-variable_set
    new_subtree = []
    if(len(variable_set)==0):
        new_subtree = str(random()*9+1)
        return set_subtree_at_path(mutant, point, new_subtree)
    elif(len(variable_set)==1):
        flag = False
        programs = []
        while(not flag):
            program, _= random_program(2, input_dim,False,variables)
            flag = len(find_variable_indices(program)) == 1
            if not flag:
                program = cleaned_program(program)
        programs.append(program)
        #Combine programs together through a binary operation
        new_subtree = programs[0]
        for i in range(1,len(programs)):
            op, arity, symbol,p = choices(OPERATIONS_BINARY,weights=BINARY_WEIGHTS, k=1)[0]
            new_subtree = [symbol] + [new_subtree, programs[i]]
    else:
        new_subtree = random_program(max_depth, input_dim,False,variables)[0]
    mutant = set_subtree_at_path(mutant, point, new_subtree)
    if(random()<0.4):
            points = get_subtree_points_recursive(mutant)
            if not points:
                return mutant
            point = choice(points)
            #Randomly choose a binary operation
            op, arity, symbol,p = choices(OPERATIONS_BINARY,weights=BINARY_WEIGHTS, k=1)[0]
            if(len(access_node_by_path(mutant, point)) == 3):
                mutant = swap_operation_at_path(mutant, point, symbol)

    return mutant

# Hoist mutation

In [264]:
def hoist_mutation(program):
    """Mutation of a program through hoist"""
    # Obtein a random subtree
    subtree, parent_info = get_subtree(program)
    
    if not isinstance(subtree, list):
        # If the subtree is a leaf, return the program, hoist cannot be done
        return program

    # Get the inner subtree
    inner_subtree, _ = get_subtree(subtree)

    if parent_info is None:
        # Sobstitute the program with the inner subtree
        return inner_subtree
    else:
        # Sobstitute the parent with the inner subtree
        parent, idx = parent_info
        parent[idx] = inner_subtree
        return program

Let's create a new tweak function that is able to tweak a program by adding an unary operation in leaf nodes (if it not a constant value nor already a unary operation).

In [265]:
def tweak_program_2(program):
    """
    Modifica un sottoalbero del programma aggiungendo un operatore unario
    su una foglia, evitando di applicarlo a una foglia già modificata da un operatore unario.
    """
    # Trova tutte le foglie del programma
    def get_leaf_indices(node, path=()):
        """
        Ritorna i percorsi alle foglie dell'albero.
        Una foglia è un valore (stringa o numero) non ulteriormente divisibile.
        """
        if isinstance(node, (str, int, float)):  # Nodo foglia (variabile o costante)
            return [path]
        elif isinstance(node, list) and len(node) > 1:  # Nodo interno valido
            indices = []
            for i, child in enumerate(node[1:], start=1):  # Salta l'operatore
                indices.extend(get_leaf_indices(child, path + (i,)))
            return indices
        return []  # Nodo vuoto o non valido

    # Ottieni tutte le foglie
    leaf_indices = get_leaf_indices(program)
    if not leaf_indices:
        return program  # Nessuna modifica possibile

    # Seleziona casualmente una foglia
    selected_leaf_path = choice(leaf_indices)

    # Accedi alla foglia selezionata
    node = program
    for idx in selected_leaf_path[:-1]:
        node = node[idx]

    # Verifica che il nodo sia valido prima di modificare
    if isinstance(node, list) and len(selected_leaf_path) > 0:
        leaf = node[selected_leaf_path[-1]]
        
        # Verifica se la foglia è modificabile
        if isinstance(leaf, (str, int, float)):
            # Scegli un operatore unario
            unary_operator = choice(["sin({})", "cos({})", "tan({})", "log({})", "sqrt({})"])
            
            # Applica l'operatore unario
            node[selected_leaf_path[-1]] = [unary_operator, leaf]
        elif isinstance(leaf, list) and len(leaf) == 2 and isinstance(leaf[0], str):
            # La foglia è già un operatore unario applicato: non fare nulla
            pass

    return program


## Crossover

We can use a croossover function that receives only 2 parents and, if one of them is a leaf program simply return casually one of the 2 programs (avoiding to perform the operation for programs with no childrens). Otherwise, select random indexes for both the parents and combine the first part of the tree with the second part of the tree of the 2 parents, returning a new individual.

In [267]:
def crossover(parent1, parent2):
    """Crossover by subtree swapping."""
    # Ottieni un sottoalbero casuale da entrambi i genitori
    subtree1, parent1_info = get_subtree(parent1)
    subtree2, _ = get_subtree(parent2)

    if parent1_info is None:
        # Sostituisci tutto parent1 con il sottoalbero di parent2
        return subtree2
    else:
        # Sostituisci il sottoalbero in parent1
        parent, idx = parent1_info
        parent[idx] = subtree2
        return parent1

## Genetic algorithm

### Data loading

In [ ]:
#load the problem with problem_X, for X that goes from 0 to 8
problem = np.load('data/problem_4.npz')
x = problem['x']
y = problem['y']
print(x.shape)
print(y.shape)

## Tournament selection for parents with fitness hole, tau=10
Tournament selection is used in order to select an individual over a population. 
This algorithm, given the population, selects 10 random individuals and performs a tournament among them. There is a probability of 90% that the fittest individual wins, while, the remaining 10% of times, the opposite is done. 
Implementing a fitness hole as described can actually be beneficial in overcoming the challenges posed by an adaptive change that requires multiple intermediate steps. By intentionally allowing less fit individuals a chance to win, fitness holes can help navigate the evolutionary pathway where direct progression is hindered by intermediate steps that reduce overall fitness. This approach ensures that even though the final adaptation is advantageous, the evolutionary path to achieve it can successfully bypass 'fitness holes' that would otherwise deselect the intermediates before the final adaptation is achieved.

In [269]:
def tournament_selection(population,tau=10):
    tau = min(tau, len(population)) #not needed in theory
    tournament_indices = np.random.choice(len(population), tau, replace=False)

    considered_individuals = []
    for index in tournament_indices:
        considered_individuals.append(population[index])
    considered_individuals.sort(key=lambda i: i.fitness)
    if random() < 0.9:
        winner = considered_individuals[0].genome
    else:
        #Select one among the second and the worst
        winner = considered_individuals[-1].genome
    return winner


## Parameters

In [270]:
generations = 200
population_size = 200
p_crossover = 0.6
p_mutation = 0.4
p_hoist = 0.1
tweak_probability = 0.2
max_depth = 2 
elite_size = 2
offspring_size = population_size 

Different EA aproach:
1. In this version we always include the elite inside the next generation as a first step
2. We extend population with the new population (resulting in having elites twice)
3. We take only the distinct individuals
4. We mantain inside the population only the population_size best individuals.

In [ ]:
# Inizializza popolazione
np.seterr(all='ignore')
input_dim = x.shape[0]
population = [Individual(genome=generate_program(input_dim)) for _ in range(population_size)]
for i in population:
    i.fitness=fitness_function(i.genome, x, y)
# Loop principale per le generazioni
def run_genetic_algorithm():
    global population

    for gen in range(generations):
        population.sort(key=lambda i: i.fitness)
        mape_val = mape(population[0].genome, x, y)
        mse_val = mse(population[0].genome, x, y)
        np.seterr(all='warn')
        print(f"Generation {gen + 1}, mape: {mape_val:.6f}, mse: {mse_val:.6f}")
        #population is already sorted, so:
        print(f"Best formula: {program_to_string(population[0].genome)}")
        np.seterr(all='ignore')
        
        # Crea la nuova generazione
        next_population = []
        next_population.extend(population[:elite_size])  # Mantieni i migliori individui
        
        
        while len(next_population) < offspring_size:
            if random() < p_crossover:
                # Crossover
                #With random choice is much faster than tournament selection
                #Choose the best and the second best parent
                parent1, parent2 = tournament_selection(population), tournament_selection(population)
                child1 = crossover(safe_copy(parent1), safe_copy(parent2))
                next_population.append(Individual(genome=child1, fitness=fitness_function(child1, x, y)))
            
                if random() < tweak_probability and len(next_population) < offspring_size:
                    new_ind = tweak_program_2(safe_copy(child1))
                    next_population.append(Individual(genome=new_ind, fitness=fitness_function(new_ind, x, y)))
            else:
                # Mutate directly a parent
                parent = tournament_selection(population)
                mutant = mutate(safe_copy(parent), input_dim)
                if random() < p_hoist:
                    mutant = hoist_mutation(safe_copy(mutant))
                    
                next_population.append(Individual(genome=mutant, fitness=fitness_function(mutant, x, y)))
                    
        # the new population is the one generated in the offspring
        population.extend(next_population)

        # Remove duplicates
        unique_population = {}
        for prog in population:
            serialized = str(prog)
            if serialized not in unique_population:
                unique_population[serialized] = prog
        
        # update fitness of the new population
        population = list(unique_population.values())
        
        population.sort(key=lambda i: i.fitness)
        population = population[:population_size]
        

    # Identify the best program
    population.sort(key=lambda i: i.fitness)
    best_program = population[0]
    best_mape = mape(best_program.genome, x, y)

    print("Best program:", best_program.genome, "; Fitness:", best_mape)
    return best_program.genome

best_program = run_genetic_algorithm()

In [ ]:
program_to_string(best_program)